In [1]:
import pandas as pd
import datetime

In [2]:
# OrderBookPlots2002_df = pd.read_csv('./resources/2002.csv')
# OrderBookPlots2002_df = OrderBookPlots2002_df[OrderBookPlots2002_df.DATE.isin(['2002-08-12', '2002-08-13'])]
# OrderBookPlots2002_df

In [3]:
# OrderBookPlots2002_df['TRADENO'] = OrderBookPlots2002_df['TRADENO'].astype(pd.Int64Dtype())

In [4]:
# OrderBookPlots2002_df.sort_values(by=['NO'], inplace=True)

In [5]:
# OrderBookPlots2002_df[(OrderBookPlots2002_df.PRICE != OrderBookPlots2002_df.TRADEPRICE) &
#                  pd.notnull(OrderBookPlots2002_df.TRADEPRICE)]

In [6]:
OrderBookPlots2007_df = pd.read_csv('./resources/LKOH_SBER_GMKN_2007_10.csv')
OrderBookPlots2007_df = OrderBookPlots2007_df[(OrderBookPlots2007_df.SECCODE == 'LKOH') & 
                                              (OrderBookPlots2007_df.DATE == '2007-10-08')]
# OrderBookPlots2007_df = OrderBookPlots2007_df[OrderBookPlots2007_df.DATE.isin(['2007-10-01', '2007-10-02'])]
# OrderBookPlots2007_df

In [7]:
OrderBookPlots2007_df['TRADENO'] = OrderBookPlots2007_df['TRADENO'].astype(pd.Int64Dtype())

In [8]:
OrderBookPlots2007_df.sort_values(by=['NO'], inplace=True)

In [9]:
OrderBookPlots2007_df[(OrderBookPlots2007_df.PRICE != OrderBookPlots2007_df.TRADEPRICE) &
                 pd.notnull(OrderBookPlots2007_df.TRADEPRICE)]

,Unnamed: 0,NO,SECCODE,BUYSELL,TIME,ORDERNO,ACTION,PRICE,VOLUME,TRADENO,TRADEPRICE,DATE,OBPLOTNO
1544302,17145,17146,LKOH,S,103001000,17122,2,2179.99,40,332168465,2180.00,2007-10-08,1399
1544318,17169,17170,LKOH,S,103001000,17071,2,2178.90,10,332168477,2180.00,2007-10-08,1399
1544321,17192,17193,LKOH,S,103001000,17081,2,2179.99,5,332168488,2180.00,2007-10-08,1399
1544324,17213,17214,LKOH,B,103001000,17130,2,0.00,50,332168499,2181.99,2007-10-08,129
1544326,17215,17216,LKOH,B,103001000,17053,2,2200.00,3,332168500,2181.99,2007-10-08,129
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834830,910304,910305,LKOH,S,174458000,395515,2,0.00,1,332355810,2175.60,2007-10-08,79324
1834832,910306,910307,LKOH,S,174458000,395515,2,0.00,84,332355811,2175.50,2007-10-08,79324
1834835,910309,910310,LKOH,S,174458000,395496,2,0.00,10,332355812,2175.50,2007-10-08,79324
1834836,910326,910327,LKOH,B,174458000,395498,2,0.00,1,332355821,2176.02,2007-10-08,79201


In [10]:
# OrderBookPlots2002_2000ms_df = OrderBookPlots2002_df[(OrderBookPlots2002_df.TIME >= 103002000) & (OrderBookPlots2002_df.TIME <= 103003000)]
# OrderBookPlots2002_2000ms_df

In [11]:
# OrderBookPlots2002_2000ms_df = OrderBookPlots2002_df[(OrderBookPlots2002_df.TIME >= 103002000) & (OrderBookPlots2002_df.TIME <= 103003000)]
def form_datetimemlls(df):
    scnd_size_df = pd.DataFrame(df.groupby('TIME').size())
    scnd_size_df.reset_index(inplace=True)
    scnd_size_df = scnd_size_df.rename(columns = {0:'SCNDSIZE'})
    df = df.merge(scnd_size_df, on='TIME')
    df['SEC_IND'] = df.groupby(['TIME']).cumcount()
    df['MILLISS'] = round(df['SEC_IND'] / df['SCNDSIZE'] * 1000)
    df['MILLISS'] = df['MILLISS'].astype(int)
    df.drop(['SCNDSIZE', 'SEC_IND'], axis=1, inplace=True)
    df['DATETIME'] = df.apply(lambda row: datetime.datetime.strptime(row.DATE, '%Y-%m-%d'), axis=1)
    df['DATETIMEMLLS'] = df.apply(lambda row: datetime.datetime(row.DATETIME.year,
                                                               row.DATETIME.month,
                                                               row.DATETIME.day,
                                                               int(row.TIME // 1e7),
                                                               int((row.TIME // 1e5) % 100),
                                                               int((row.TIME // 1e3) % 100)),
                                  axis=1)
    df['DATETIMEMLLS'] = df.apply(lambda row: row.DATETIMEMLLS +
                                 datetime.timedelta(milliseconds=row.MILLISS),
                                 axis=1)
    df.drop(['DATETIME', 'MILLISS', 'TIME'], axis=1, inplace=True)
    return(df)
# OrderBookPlots2002_2000ms_df = form_datetimemlls(OrderBookPlots2002_2000ms_df)
# OrderBookPlots2002_2000ms_df

In [12]:
# OrderBookPlots2002_df = form_datetimemlls(OrderBookPlots2002_df)

In [13]:
OrderBookPlots2007_df = form_datetimemlls(OrderBookPlots2007_df)

In [14]:
class OBPlots2OBPlots_Attributes:
    def __init__(self, OBPs_df):
        self.OBPs_df = OBPs_df
        self.secs = pd.unique(self.OBPs_df.SECCODE)
        self.dates = pd.unique(self.OBPs_df.DATE)
#         print(self.dates)
        self.obplotnos = pd.unique(self.OBPs_df.OBPLOTNO)
        self.obp_atts_dic = {
            sec:datedic for (sec, datedic) in zip(
                self.secs,
                [{
                    date:obps_attsdic for (date, obps_attsdic) in zip(
                        self.dates, 
                        [{} for i in range(len(self.dates))]
                    )
                } for i in range(len(self.secs))]
            )
        }
        self.iterate_ob_rows()
    
    # Accordin to specs ACTION 0 shows balance of order, not cancelled volune of the order, IT'S WRONG
    # Actually, it shows cancelled volune of the order
    def ob2curatt(self, SECCODE, DATE, OBPLOTNO, BUYSELL, ACTION, PRICE, VOLUME, TRADEPRICE):
        if (BUYSELL == 'B') & (ACTION == 0):
            CBOVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['CBOVOL'] + (PRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['CBOVOL'] = CBOVOL
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['SHAREBAL'] -= VOLUME
            return({'ATT': 'CBOVOL', 'VAL': CBOVOL})
        elif (BUYSELL == 'S') & (ACTION == 0):
            CSOVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['CSOVOL'] + (PRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['CSOVOL'] = CSOVOL
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['SHAREBAL'] += VOLUME
            return({'ATT': 'CSOVOL', 'VAL': CSOVOL})
        elif (BUYSELL == 'B') & (ACTION == 1):
            BOVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['BOVOL'] + (PRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['BOVOL'] = BOVOL
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['SHAREBAL'] += VOLUME
            return({'ATT': 'BOVOL', 'VAL': BOVOL})
        elif (BUYSELL == 'S') & (ACTION == 1):
            SOVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['SOVOL'] + (PRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['SOVOL'] = SOVOL
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['SHAREBAL'] -= VOLUME
            return({'ATT': 'SOVOL', 'VAL': SOVOL})
        elif (BUYSELL == 'B') & (ACTION == 2):
            BTVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['BTVOL'] + (TRADEPRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['BTVOL'] = BTVOL
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['SHAREBAL'] -= VOLUME
            return({'ATT': 'BTVOL', 'VAL': BTVOL})
        elif (BUYSELL == 'S') & (ACTION == 2):
            STVOL = self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['STVOL'] + (TRADEPRICE * VOLUME)
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV']['STVOL'] = STVOL
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['SHAREBAL'] += VOLUME
            return({'ATT': 'STVOL', 'VAL': STVOL})
        
    def ob_row2obp_atts(self, SECCODE, DATE, NO, BUYSELL, ORDERNO,
                        ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO,
                        DATETIMEMLLS):
        if OBPLOTNO not in self.obp_atts_dic[SECCODE][DATE]:
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO] = {}
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['PREV'] = {
                'CBOVOL': 0, # cancel buy order volume
                'CSOVOL': 0, # cancel sell order volume
                'BOVOL': 0, # buy order volume
                'SOVOL': 0, # sell order volume
                'BTVOL': 0, # buy trade volumne
                'STVOL': 0 # sell trade volume
            }
            self.obp_atts_dic[SECCODE][DATE][OBPLOTNO]['SHAREBAL'] = 0
            
        self.obp_atts_dic[SECCODE][DATE][OBPLOTNO][NO] = self.ob2curatt(SECCODE, DATE, OBPLOTNO, BUYSELL, ACTION, 
                                                                    PRICE, VOLUME, TRADEPRICE)
        self.obp_atts_dic[SECCODE][DATE][OBPLOTNO][NO]['DATETIMEMLLS'] = DATETIMEMLLS
#         print(SECCODE)
#         print(self.obp_atts[SECCODE])
        
    def iterate_ob_rows(self):
        [self.ob_row2obp_atts(SECCODE, DATE, NO, BUYSELL, ORDERNO, 
                              ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO,
                              DATETIMEMLLS)
        for (SECCODE, DATE, NO, BUYSELL, ORDERNO,
             ACTION, PRICE, VOLUME, TRADEPRICE, OBPLOTNO,
             DATETIMEMLLS)
         in zip(self.OBPs_df.SECCODE,
                self.OBPs_df.DATE,
                self.OBPs_df.NO,
                self.OBPs_df.BUYSELL,
                self.OBPs_df.ORDERNO,
                self.OBPs_df.ACTION,
                self.OBPs_df.PRICE,
                self.OBPs_df.VOLUME,
                self.OBPs_df.TRADEPRICE,
                self.OBPs_df.OBPLOTNO,
                self.OBPs_df.DATETIMEMLLS)]

In [15]:
# obp_atts_2000ms = OBPlots2OBPlots_Attributes(OrderBookPlots2002_2000ms_df)

In [16]:
def obp_atts2order_atts_df(obp_atts):
    order_atts_dic = {}
    for SECCODE in obp_atts:
        for DATE in obp_atts[SECCODE]:
            for OBPLOTNO in obp_atts[SECCODE][DATE]:
                for NO in obp_atts[SECCODE][DATE][OBPLOTNO]:
                    order_atts_dic[NO] = {}
                    if (NO != 'PREV') & (NO != 'SHAREBAL'):
#                         print(NO)
#                         print(obp_atts[SECCODE][DATE][OBPLOTNO][NO])
                        order_atts_dic[NO] = obp_atts[SECCODE][DATE][OBPLOTNO][NO]
                        order_atts_dic[NO]['NO'] = NO
                        order_atts_dic[NO]['OBPLOTNO'] = OBPLOTNO
                        order_atts_dic[NO]['DATE'] = DATE
                        order_atts_dic[NO]['SECCODE'] = SECCODE
                        order_atts_dic[NO]['SHAREBAL'] = obp_atts[SECCODE][DATE][OBPLOTNO]['SHAREBAL']

    order_atts_df = pd.DataFrame.from_dict(order_atts_dic, "index")
    order_atts_df.sort_values(by=['DATETIMEMLLS', 'NO'], inplace=True)
    return(order_atts_df)

In [17]:
# obp_atts_2000ms_df = obp_atts2order_atts_df(obp_atts_2000ms.obp_atts_dic)

In [18]:
# obp_atts_2002 = OBPlots2OBPlots_Attributes(OrderBookPlots2002_df)
# obp_atts_2002_df = obp_atts2order_atts_df(obp_atts_2002.obp_atts_dic)
# obp_atts_2002_df

In [19]:
obp_atts_2007 = OBPlots2OBPlots_Attributes(OrderBookPlots2007_df)
obp_atts_2007_df = obp_atts2order_atts_df(obp_atts_2007.obp_atts_dic)
obp_atts_2007_df

,ATT,VAL,DATETIMEMLLS,NO,OBPLOTNO,DATE,SECCODE,SHAREBAL
13,SOVOL,14649812.50,2007-10-08 10:30:00.000,13,0,2007-10-08,LKOH,0
46,SOVOL,33210.00,2007-10-08 10:30:00.001,46,2,2007-10-08,LKOH,-15
74,BOVOL,99500.00,2007-10-08 10:30:00.003,74,4,2007-10-08,LKOH,50
76,SOVOL,22000.00,2007-10-08 10:30:00.004,76,6,2007-10-08,LKOH,-10
77,BOVOL,12660.00,2007-10-08 10:30:00.005,77,8,2007-10-08,LKOH,0
...,...,...,...,...,...,...,...,...
910328,STVOL,2176.02,2007-10-08 17:44:58.812,910328,79201,2007-10-08,LKOH,-338
910329,STVOL,87067.32,2007-10-08 17:44:58.875,910329,79201,2007-10-08,LKOH,-338
910330,BTVOL,87067.32,2007-10-08 17:44:58.938,910330,79201,2007-10-08,LKOH,-338
910337,CSOVOL,21774.40,2007-10-08 17:44:59.000,910337,78407,2007-10-08,LKOH,0


In [20]:
obp_atts_2007_df = obp_atts_2007_df[obp_atts_2007_df.SHAREBAL == 0]
obp_atts_2007_df

,ATT,VAL,DATETIMEMLLS,NO,OBPLOTNO,DATE,SECCODE,SHAREBAL
13,SOVOL,14649812.50,2007-10-08 10:30:00.000,13,0,2007-10-08,LKOH,0
77,BOVOL,12660.00,2007-10-08 10:30:00.005,77,8,2007-10-08,LKOH,0
208,BOVOL,1983.67,2007-10-08 10:30:00.009,208,14,2007-10-08,LKOH,0
220,BOVOL,4110.62,2007-10-08 10:30:00.019,220,30,2007-10-08,LKOH,0
227,BOVOL,8600.00,2007-10-08 10:30:00.027,227,42,2007-10-08,LKOH,0
...,...,...,...,...,...,...,...,...
910247,STVOL,7254644.35,2007-10-08 17:44:57.875,910247,79171,2007-10-08,LKOH,0
910248,BTVOL,7309044.35,2007-10-08 17:44:57.917,910248,79171,2007-10-08,LKOH,0
910249,STVOL,7309044.35,2007-10-08 17:44:57.958,910249,79171,2007-10-08,LKOH,0
910337,CSOVOL,21774.40,2007-10-08 17:44:59.000,910337,78407,2007-10-08,LKOH,0


In [24]:
def obp_atts2obp_atts_df(order_atts_df):
    def buy_sell_obp(gr):
        buy_minus_sell_vol = gr[gr.ATT == 'BOVOL']['VAL'].sum() - gr[gr.ATT == 'SOVOL']['VAL'].sum()
        if buy_minus_sell_vol > 0:
            return('B')
        elif buy_minus_sell_vol < 0:
            return('S')
        else:
            return('N')
    def trades_notrades(gr):
        trades_count = gr[gr.ATT == 'BTVOL'].size + gr[gr.ATT == 'STVOL'].size
        if trades_count == 0:
            return('T')
        else:
            return('NT')
    buy_sell_obp_ser = order_atts_df.groupby(['DATE', 'SECCODE', 'OBPLOTNO']).apply(lambda gr: buy_sell_obp(gr))
    trades_notrades_ser = order_atts_df.groupby(['DATE', 'SECCODE', 'OBPLOTNO']).apply(lambda gr: trades_notrades(gr))
    df = pd.DataFrame.from_dict({'BUYSELLOBP': buy_sell_obp_ser, 'TRADESNOTRADES': trades_notrades_ser})
    df.reset_index(inplace=True)
    return(df)

In [25]:
obp_atts_2007_df = obp_atts2obp_atts_df(obp_atts_2007_df)
obp_atts_2007_df

,DATE,SECCODE,OBPLOTNO,BUYSELLOBP,TRADESNOTRADES
0,2007-10-08,LKOH,0,S,T
1,2007-10-08,LKOH,8,B,T
2,2007-10-08,LKOH,14,B,T
3,2007-10-08,LKOH,30,B,T
4,2007-10-08,LKOH,42,B,T
...,...,...,...,...,...
16864,2007-10-08,LKOH,79177,S,T
16865,2007-10-08,LKOH,79179,S,T
16866,2007-10-08,LKOH,79187,B,NT
16867,2007-10-08,LKOH,79238,S,T


In [26]:
obp_atts_2007_df[(obp_atts_2007_df.BUYSELLOBP != 'N') & (obp_atts_2007_df.TRADESNOTRADES != 'NT')]

,DATE,SECCODE,OBPLOTNO,BUYSELLOBP,TRADESNOTRADES
693,2007-10-08,LKOH,3744,N,NT
967,2007-10-08,LKOH,5278,N,NT
1157,2007-10-08,LKOH,6103,N,NT
1357,2007-10-08,LKOH,7096,N,NT
1378,2007-10-08,LKOH,7200,N,NT
...,...,...,...,...,...
16706,2007-10-08,LKOH,77874,N,NT
16737,2007-10-08,LKOH,78058,N,NT
16791,2007-10-08,LKOH,78536,N,NT
16855,2007-10-08,LKOH,79041,N,NT


In [27]:
obp_atts_2007_df[obp_atts_2007_df.TRADESNOTRADES == 'NT']

,DATE,SECCODE,OBPLOTNO,BUYSELLOBP,TRADESNOTRADES
5,2007-10-08,LKOH,46,B,NT
8,2007-10-08,LKOH,52,B,NT
9,2007-10-08,LKOH,60,B,NT
10,2007-10-08,LKOH,64,S,NT
11,2007-10-08,LKOH,68,B,NT
...,...,...,...,...,...
16856,2007-10-08,LKOH,79049,B,NT
16860,2007-10-08,LKOH,79078,N,NT
16862,2007-10-08,LKOH,79171,S,NT
16866,2007-10-08,LKOH,79187,B,NT
